<a href="https://colab.research.google.com/github/aryamanpathak2022/Cnn_models/blob/main/CNN_C10_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORTS


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



# Data augmentation and normalization for training, normalization for testing


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 training and test datasets


In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane','car', 'bird', 'cat','deer', 'dog','frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:03<00:00, 46622827.39it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#subset of training

In [ ]:
# from torch.utils.data import Subset
# subset_indices = list(range(1000))
# train_subset = Subset(trainset, subset_indices)
# trainloader = torch.utils.data.DataLoader(train_subset, batch_size=100, shuffle=True, num_workers=2)

# Define the CNN architecture

In [ ]:
class ComplexCNN(nn.Module):
     def __init__(self):
        super(ComplexCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1) # (N, 3, 32, 32) -> (N, 64, 32, 32)
        self.bn1 = nn.BatchNorm2d(64)                          # (N, 64, 32, 32) -> (N, 64, 32, 32)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1) # (N, 64, 32, 32) -> (N, 128, 32, 32)
        self.bn2 = nn.BatchNorm2d(128)                         # (N, 128, 32, 32) -> (N, 128, 32, 32)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1) # (N, 128, 32, 32) -> (N, 256, 32, 32)
        self.bn3 = nn.BatchNorm2d(256)                         # (N, 256, 32, 32) -> (N, 256, 32, 32)
        self.pool = nn.MaxPool2d(2, 2)                         # (N, 256, 32, 32) -> (N, 256, 16, 16)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)                # (N, 256 * 4 * 4) -> (N, 1024)
        self.fc2 = nn.Linear(1024, 512)                        # (N, 1024) -> (N, 512)
        self.fc3 = nn.Linear(512, 10)                          # (N, 512) -> (N, 10)
        self.dropout = nn.Dropout(0.5)

     def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))         # (N, 3, 32, 32) -> (N, 64, 32, 32) -> (N, 64, 16, 16)
        x = self.pool(F.relu(self.bn2(self.conv2(x))))         # (N, 64, 16, 16) -> (N, 128, 16, 16) -> (N, 128, 8, 8)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))         # (N, 128, 8, 8) -> (N, 256, 8, 8) -> (N, 256, 4, 4)
        x = x.view(-1, 256 * 4 * 4)                            # (N, 256, 4, 4) -> (N, 256 * 4 * 4)
        x = F.relu(self.fc1(x))                                # (N, 256 * 4 * 4) -> (N, 1024)
        x = self.dropout(x)                                    # (N, 1024) -> (N, 1024)
        x = F.relu(self.fc2(x))                                # (N, 1024) -> (N, 512)
        x = self.fc3(x)                                        # (N, 512) -> (N, 10)
        return x

# Initialize the model, loss function, and optimizer

In [ ]:
net = ComplexCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Training

In [ ]:
for epoch in range(10):  # loop over the dataset 10 times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

Finished Training


# Testing

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 1000 test images: %d %%' % (100 * correct / total))


Accuracy of the network on the 1000 test images: 39 %


#saving the model


In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
model = ComplexCNN()
model.load_state_dict(torch.load('cifar_net.pth'))

# Iterate over the state dictionary and print the weights
state_dict = model.state_dict()
for name, param in state_dict.items():
    print(f"{name}: {param}")

conv1.weight: tensor([[[[ 0.1835,  0.1329,  0.0645],
          [ 0.1492, -0.1487,  0.0706],
          [ 0.1759,  0.1469,  0.0667]],

         [[-0.1039, -0.0481,  0.1380],
          [ 0.0619, -0.0117,  0.1013],
          [ 0.0168, -0.0220, -0.1348]],

         [[-0.1675,  0.0320, -0.0239],
          [ 0.0577,  0.0104,  0.1043],
          [ 0.0170, -0.1850,  0.0699]]],


        [[[ 0.0886, -0.0011,  0.1651],
          [-0.0457,  0.0688,  0.1519],
          [-0.0562, -0.0739,  0.0738]],

         [[ 0.1787, -0.1839, -0.1408],
          [-0.0570,  0.0552,  0.0362],
          [ 0.0384, -0.1706,  0.1676]],

         [[ 0.1610, -0.0071, -0.0323],
          [-0.1691, -0.0034,  0.0447],
          [-0.1493,  0.0684, -0.1478]]],


        [[[ 0.1788, -0.1822,  0.0126],
          [-0.0454,  0.1205, -0.1645],
          [-0.1128,  0.0410,  0.1085]],

         [[-0.1783,  0.0867, -0.0751],
          [-0.1364, -0.1701,  0.0534],
          [ 0.1318,  0.1650, -0.1836]],

         [[ 0.1892, -0.1621, -